# Super Bowl flights: 2022

### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import requests
import matplotlib as mpl
import geopandas as gpd
import json
import io
import os
import glob
import numpy as np
from shapely.geometry import Point, LineString

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Before we start, metadata from FAA

In [4]:
registration = pd.read_csv(
    "/Users/stiles/data/aviation/master_faa_owners_database.csv", low_memory=False
)
registration.rename(columns={"Unnamed: 0": "id", "n_number": "reg"}, inplace=True)

In [5]:
registration.drop(["unnamed:_13"], axis=1, inplace=True)

In [6]:
registration.rename(
    columns={"mfr": "manufacturer", "name_y": "owner_original", "clean_name": "owner"},
    inplace=True,
)

### Filter for commercial and cargo

In [130]:
airlines = [
    "DELTA AIR LINES INC",
    "AMERICAN AIRLINES INC",
    "SKYWEST AIRLINES INC",
    "SWIFT AIR LLC",
    "UNITED AIRLINES INC",
    "SOUTHWEST AIRLINES CO",
    "JETBLUE AIRWAYS CORP",
    "HORIZON AIR INDUSTRIES",
    "ALASKA AIRLINES INC",
    "REPUBLIC AIRWAYS INC",
    "SPIRIT AIRLINES INC",
    "MESA AIRLINES INC",
]

In [131]:
airlines

['DELTA AIR LINES INC',
 'AMERICAN AIRLINES INC',
 'SKYWEST AIRLINES INC',
 'SWIFT AIR LLC',
 'UNITED AIRLINES INC',
 'SOUTHWEST AIRLINES CO',
 'JETBLUE AIRWAYS CORP',
 'HORIZON AIR INDUSTRIES',
 'ALASKA AIRLINES INC',
 'REPUBLIC AIRWAYS INC',
 'SPIRIT AIRLINES INC',
 'MESA AIRLINES INC']

In [132]:
cargo = [
    "UNITED PARCEL SERVICE CO",
    "FEDERAL EXPRESS CORP",
    "FEDERAL EXPRESS CORPORATION",
    "CARGO AIRCRAFT MANAGEMENT INC",
    "ATLAS AIR INC",
]

In [133]:
cargo

['UNITED PARCEL SERVICE CO',
 'FEDERAL EXPRESS CORP',
 'FEDERAL EXPRESS CORPORATION',
 'CARGO AIRCRAFT MANAGEMENT INC',
 'ATLAS AIR INC']

In [134]:
planes = registration[
    (~registration["name"].isin(airlines))
    & (~registration["name"].isin(cargo))
    & (registration["type_aircraft"].str.contains("4|5"))
    & (registration["certification"].str.contains("1T"))
]

### Download data from Flightradar24

In [8]:
os.chdir("data/")

In [9]:
# !wget --user LAX_Super_Bowl --password GEpQyE16da -r -np –quiet -nH --cut-dirs=3 -R index.html 'https://secure.flightradar24.com/LAX_Super_Bowl/' -q

In [10]:
# !unzip -qq \*.zip

In [11]:
# !rm -f *.zip
# !rm -f *.tmp

In [12]:
# !mkdir positions
# !mkdir flights

In [13]:
# !mv -f *flights.csv flights

In [14]:
# !mv -f *.csv positions

---

## Process 'positions' data showing each point along a flight

### Set path for positions and define the files we'll concatenate

In [92]:
a_position = pd.read_csv("positions/20210221_652101354.csv")
a_position.head()

,snapshot_id,altitude,heading,latitude,longitude,radar_id,speed,squawk
0,1613865603,30250,165,35.23273,-119.48128,41012,493,2152
1,1613865612,30000,165,35.21644,-119.47603,2818,490,2152
2,1613865703,26875,165,35.01706,-119.41240,2818,480,2152
3,1613865765,24625,165,34.88621,-119.37111,2685,470,2152
4,1613865827,22950,156,34.75795,-119.32581,42232,464,2152


In [16]:
path = "positions"
files = glob.glob(os.path.join(path, "*.csv"))

### Read the csv and create a 'flightid' field so we can track unique flights

In [17]:
file_df = (
    pd.read_csv(f, encoding="ISO-8859-1", low_memory=False).assign(
        flightid=os.path.basename(f)
    )
    for f in files
)

### Concateate the frames

In [18]:
positions_df = pd.concat(file_df, ignore_index=True)
positions_df.head()

,snapshot_id,altitude,heading,latitude,longitude,radar_id,speed,squawk,flightid
0,1644108124,34000,218,63.62736,-91.65028,40302,492,0,20220206_716709931.csv
1,1644108186,34000,218,63.51759,-91.84877,40302,491,2225,20220206_716709931.csv
2,1644108248,34000,218,63.40942,-92.04240,40302,490,2225,20220206_716709931.csv
3,1644108310,34000,218,63.29919,-92.23688,40302,490,2225,20220206_716709931.csv
4,1644108372,34000,218,63.18846,-92.43021,40302,490,2225,20220206_716709931.csv


In [19]:
len(positions_df)

9902344

### Combined our newly processed flight positions

In [20]:
positions_df["flightid"] = positions_df["flightid"].str.replace(".csv", "", regex=False)

### Split the flightid field so we have a date string to convert later and also a flightid

In [21]:
positions_df[["datestr", "flight_id"]] = positions_df.flightid.str.split(
    "_",
    expand=True,
)

In [22]:
positions_df.head()

,snapshot_id,altitude,heading,latitude,longitude,radar_id,speed,squawk,flightid,datestr,flight_id
0,1644108124,34000,218,63.62736,-91.65028,40302,492,0,20220206_716709931,20220206,716709931
1,1644108186,34000,218,63.51759,-91.84877,40302,491,2225,20220206_716709931,20220206,716709931
2,1644108248,34000,218,63.40942,-92.04240,40302,490,2225,20220206_716709931,20220206,716709931
3,1644108310,34000,218,63.29919,-92.23688,40302,490,2225,20220206_716709931,20220206,716709931
4,1644108372,34000,218,63.18846,-92.43021,40302,490,2225,20220206_716709931,20220206,716709931


### Process the 'datestr' field into something we can use

In [23]:
positions_df["date"] = pd.to_datetime(positions_df["datestr"], format="%Y%m%d")

In [24]:
positions_df["month"] = positions_df["date"].dt.month
positions_df["day"] = positions_df["date"].dt.day
positions_df["weekday"] = positions_df["date"].dt.day_name()

### Convert the unix timestampt to human datetime and localize

In [25]:
positions_df["date_time"] = pd.to_datetime(positions_df["snapshot_id"], unit="s")
positions_df["utc_datetime"] = pd.to_datetime(
    positions_df["date_time"], format="%Y-%m-%dT%H:%M:%SZ"
).dt.tz_localize("UTC")

In [26]:
positions_df["datetime_pst"] = positions_df["utc_datetime"].dt.tz_convert(
    "America/Los_Angeles"
)

In [27]:
positions_df["date"] = pd.to_datetime(positions_df["datetime_pst"]).dt.strftime(
    "%m/%d/%Y"
)
positions_df["time"] = pd.to_datetime(positions_df["datetime_pst"]).dt.strftime(
    "%H:%M:%S"
)
positions_df["display_time"] = pd.to_datetime(positions_df["datetime_pst"]).dt.strftime(
    "%I:%M %p"
)

In [28]:
positions_df = positions_df.drop(
    [
        "snapshot_id",
        "radar_id",
        "day",
        "datestr",
        "utc_datetime",
        "date_time",
        "datetime_pst",
        "display_time",
    ],
    axis=1,
)

In [29]:
positions = pd.DataFrame(positions_df)

In [30]:
positions.sort_values(by="date", ascending=True).head()

,altitude,heading,latitude,longitude,speed,squawk,flightid,flight_id,date,month,weekday,time
0,34000,218,63.62736,-91.65028,492,0,20220206_716709931,716709931,02/05/2022,2,Sunday,16:42:04
3587371,10850,115,34.09784,-118.83785,271,3232,20220206_716777068,716777068,02/05/2022,2,Sunday,19:40:12
3587370,10850,115,34.10440,-118.85451,271,3232,20220206_716777068,716777068,02/05/2022,2,Sunday,19:40:00
3587369,10850,115,34.11055,-118.87041,271,3232,20220206_716777068,716777068,02/05/2022,2,Sunday,19:39:48
3587368,10850,115,34.11745,-118.88805,270,3232,20220206_716777068,716777068,02/05/2022,2,Sunday,19:39:36


---

## Process 'flights' metadata about each set of points

### Set path for flights and define the files we'll concatenate

In [93]:
a_flight = pd.read_csv("flights/20210221_flights.csv")

In [94]:
a_flight.head()

,flight_id,aircraft_id,reg,equip,callsign,flight,schd_from,schd_to,real_to,reserved
0,652101354,4960650,TCLLJ,B789,THY9,TK9,IST,LAX,LAX,NaN
1,652117615,3428875,ECMLB,A332,IBE6177,IB6177,MAD,LAX,LAX,NaN
2,652114885,7701797,RPC3501,A359,PAL102,PR102,MNL,LAX,LAX,NaN
3,652115990,7785913,9VSMY,A359,SIA38,SQ38,SIN,LAX,LAX,NaN
4,652103921,11101202,N704GT,B77L,SOO8028,9S8028,LAX,ICN,ICN,NaN


In [95]:
path = "flights"
files = glob.glob(os.path.join(path, "*.csv"))

### Read the csv and create a 'date' field

In [97]:
file_df = (
    pd.read_csv(f, encoding="ISO-8859-1", low_memory=False).assign(
        date=os.path.basename(f)
    )
    for f in files
)

### Combined our newly processed flight files

In [98]:
flights_df = pd.concat(file_df, ignore_index=True)

In [99]:
flights_df.head()

,flight_id,aircraft_id,reg,equip,callsign,flight,schd_from,schd_to,real_to,reserved,date
0,652714535,4960644,TCLLD,B789,THY9,TK9,IST,LAX,LAX,NaN,20210225_flights.csv
1,652728084,7785910,9VSMV,A359,SIA38,SQ38,SIN,LAX,LAX,NaN,20210225_flights.csv
2,652722286,7701797,RPC3501,A359,PAL102,PR102,MNL,LAX,LAX,NaN,20210225_flights.csv
3,652743424,4344683,MLWCW,GL7T,MLWCW,NaN,ZRH,LAX,LAX,NaN,20210225_flights.csv
4,652733465,9015460,B18772,B77L,CAL5155,CI5155,LAX,KIX,KIX,NaN,20210225_flights.csv


### Clean up our dates for use later

In [100]:
flights_df["date"] = flights_df["date"].str.replace("_flights.csv", "", regex=False)

In [121]:
flights_df["date"] = pd.to_datetime(flights_df.date, format="%Y%m%d")
flights_df["month"] = flights_df["date"].dt.month
flights_df["day"] = flights_df["date"].dt.day
flights_df["weekday"] = flights_df["date"].dt.day_name()
flights_df["year"] = flights_df["date"].dt.year

### Create a new dataframe with flights and export to CSV

In [122]:
flights = pd.DataFrame(flights_df)

In [124]:
flights.head()

,flight_id,aircraft_id,reg,equip,callsign,flight,schd_from,schd_to,real_to,reserved,date,month,day,weekday,year
0,652714535,4960644,TCLLD,B789,THY9,TK9,IST,LAX,LAX,NaN,2021-02-25,2,25,Thursday,2021
1,652728084,7785910,9VSMV,A359,SIA38,SQ38,SIN,LAX,LAX,NaN,2021-02-25,2,25,Thursday,2021
2,652722286,7701797,RPC3501,A359,PAL102,PR102,MNL,LAX,LAX,NaN,2021-02-25,2,25,Thursday,2021
3,652743424,4344683,MLWCW,GL7T,MLWCW,NaN,ZRH,LAX,LAX,NaN,2021-02-25,2,25,Thursday,2021
4,652733465,9015460,B18772,B77L,CAL5155,CI5155,LAX,KIX,KIX,NaN,2021-02-25,2,25,Thursday,2021


In [125]:
flights.to_csv("../data/all_flights.csv")

In [126]:
len(flights)

2022-02-14    3791
2022-02-11    3770
2022-02-12    3663
2022-02-10    3411
2022-02-06    3346
2022-02-13    3307
2022-02-08    2955
2022-02-07    2885
2022-02-09    2837
2021-03-01    1609
2021-02-26    1545
2021-02-25    1513
2021-02-27    1508
2021-02-22    1494
2021-02-28    1470
2021-02-23    1436
2021-02-21    1413
2021-02-24    1380
Name: date, dtype: int64

In [135]:
flights_reg = flights.merge(planes, on="reg")

In [144]:
flights_reg[flights_reg["no_seats"] < 25].head(10)

,flight_id,aircraft_id,reg,equip,callsign,flight,schd_from,schd_to,real_to,reserved,date,month,day,weekday,year,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,state,zip_code,region,county,country,last_action_date,cert_issue_date,certification,type_aircraft,type_engine,status_code,mode_s_code,fract_owner,air_worth_date,other_names1,other_names2,other_names3,other_names4,other_names5,expiration_date,unique_id,kit_mfr,kit_model,mode_s_code_hex,unnamed:_34,code,manufacturer,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,tc_data_sheet,tc_data_holder,type
22,652764199,11172822,N775CK,B77L,CKS259,K4259,LAX,SYD,SYD,NaN,2021-02-25,2,25,Thursday,2021,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
23,718037868,11172822,N775CK,B77L,CKS246,K4246,LAX,SYD,SYD,NaN,2022-02-13,2,13,Sunday,2022,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
24,652519647,11172822,N775CK,B77L,CKS258A,K4258A,NGO,LAX,LAX,NaN,2021-02-23,2,23,Tuesday,2021,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
25,652594912,11172822,N775CK,B77L,CKS258A,K4258A,LAX,MIA,MIA,NaN,2021-02-23,2,23,Tuesday,2021,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
26,653036217,11172822,N775CK,B77L,CKS258,K4258,NGO,LAX,LAX,NaN,2021-02-26,2,26,Friday,2021,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
27,653120727,11172822,N775CK,B77L,CKS258,K4258,LAX,MIA,MIA,NaN,2021-02-26,2,26,Friday,2021,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
28,717803390,11172822,N775CK,B77L,CKS258D,NaN,NGO,LAX,LAX,NaN,2022-02-12,2,12,Saturday,2022,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
29,717927393,11172822,N775CK,B77L,CKS258D,NaN,LAX,CVG,CVG,NaN,2022-02-12,2,12,Saturday,2022,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
30,717974693,11172822,N775CK,B77L,CKS246,K4246,CVG,LAX,LAX,NaN,2022-02-12,2,12,Saturday,2022,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR STE 400,,SALT LAKE CITY,UT,841215058,S,035,US,20201030,20201030,1T,5,5,V,52475726,,20201028,,,,,,20231031,1326317,,,AA7BD6,NaN,138527B,BOEING,777F,5,5,1,0,2,13,CLASS 3,0,,,Fixed wing multi engine
31,718037868,11172822,N775CK,B77L,CKS246,K4246,LAX,SYD,SYD,NaN,2022-02-12,2,12,Saturday,2022,66087,138527B,30046,2020,3.0,UMB BANK NA TRUSTEE,6440 S MILLROCK DR ST

### Group by flight ID to associate each flight with an aircraft

In [42]:
flight_id_grouped = (
    flights.groupby(["flight_id", "reg"]).agg("size").reset_index(name="count")
)
flight_id_grouped = flight_id_grouped.drop(["count"], axis=1)

In [43]:
len(flight_id_grouped)

28017

In [44]:
flight_id_grouped.head()

,flight_id,reg
0,652101354,TCLLJ
1,652103921,N704GT
2,652114885,RPC3501
3,652115990,9VSMY
4,652117615,ECMLB


---

### Merge to add aircraft ID and registration N number to each position

In [89]:
flights["flight_id"] = flights["flight_id"].astype(str)

In [90]:
positions = positions.merge(flights, on="flight_id")

In [91]:
positions = gpd.GeoDataFrame(positions.merge(registration, on="reg"))

KeyError: 'reg'

In [ ]:
len(positions)

---

## Geography

### Convert to positions to a GeoDataFrame using lon/lat for each point in the flight

In [51]:
geometry = [Point(xy) for xy in zip(positions.longitude, positions.latitude)]
positions_gdf = gpd.GeoDataFrame(positions, geometry=geometry)

In [52]:
positions_gdf.crs = "epsg:4326"

In [ ]:
# n_numbers = positions_geo.groupby(['reg']).agg('size').reset_index(name='count')

In [ ]:
# choppers_list = n_numbers['reg'].tolist()

In [ ]:
# n_numbers = []
# for n in choppers_list:
#     n_numbers.append(dict(n_number = n))

In [ ]:
# df = pd.DataFrame()

# for l in n_numbers:
#     n = l['n_number']
#     aircraft = positions_geo[positions_geo['n_number'] == n]
#     aircraft.to_file(f'/Users/mhustiles/data/data/helicopters/' + n + '.geojson', driver='GeoJSON')

In [ ]:
!tippecanoe --generate-ids --force -Z8 -z11 -r1 -pk -pf -o \
/Users/mhustiles/data/data/helicopters/N661PD.mbtiles \
/Users/mhustiles/data/data/helicopters/N661PD.geojson

---

### We need flight paths. Convert to points to linestring

In [58]:
positions_lines_gdf = gpd.GeoDataFrame(
    (
        positions_gdf.groupby(["flight_id", "reg"])
        .agg(
            {"geometry": lambda x: LineString(x.tolist()) if x.size > 2 else x.tolist()}
        )
        .reset_index()
    )
)

In [77]:
positions_lines_gdf = positions_lines_gdf.loc[
    positions_lines_gdf.geom_type == "LineString"
]

In [78]:
positions_lines_gdf.to_file(
    "/Users/stiles/data/aviation/positions_lines_gdf.geojson", driver="GeoJSON"
)

In [ ]:
# !tippecanoe --generate-ids --force -Z8 -z11 -r1 -pk -pf -o \
# /Users/stiles/data/aviation/positions_geo_line.mbtiles \
# /Users/stiles/data/aviation/positions_geo_line.geojson

### Merge FAA with each aircraft in our fligts, positions frames

In [79]:
positions_reg = positions_lines_gdf.merge(registration, on="reg")

In [80]:
positions_reg.to_file(
    "/Users/stiles/data/aviation/positions_reg.geojson", driver="GeoJSON"
)

In [150]:
positions_reg[positions_reg["reg"] == "N1955M"]

,flight_id,reg,geometry,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,state,zip_code,region,county,country,last_action_date,cert_issue_date,certification,type_aircraft,type_engine,status_code,mode_s_code,fract_owner,air_worth_date,other_names1,other_names2,other_names3,other_names4,other_names5,expiration_date,unique_id,kit_mfr,kit_model,mode_s_code_hex,unnamed:_34,code,manufacturer,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,tc_data_sheet,tc_data_holder,type


In [83]:
elon = positions_reg[positions_reg["reg"] == "N898TS"]

In [84]:
elon.to_file("/Users/stiles/data/aviation/positions_reg_elon.geojson", driver="GeoJSON")